In [1]:
import urllib3
import re
import os
import justext
from bs4 import BeautifulSoup
from pprint import pprint

In [2]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
user_agent = { 'user-agent': 'Mozilla/5.0 (X11; OpenBSD amd64; rv:28.0) Gecko/20100101 Firefox/28.' }

http = urllib3.PoolManager(10, headers=user_agent)

In [3]:
class Crawler:
    
    def __init__(self, corpus_path, seed_url, url_pattern, max_files=50):
        self.corpus_path = corpus_path
        self.max_files = max_files
        self.seed_url = seed_url
        self.url_pattern = url_pattern
        self.visited_links = {}
        self.to_be_visited = []
        
        if not os.path.exists(self.corpus_path):
            os.makedirs(self.corpus_path)
        
    def crawl(self):
        first_urls = self.get_page(self.seed_url)
        self.add_links(first_urls)
        next_link = self.get_next_link()

        file_counter = 1
        while next_link and file_counter < self.max_files:
            links = self.get_page(next_link)
            self.add_links(links)
            next_link = self.get_next_link()
            file_counter += 1
    
    def get_page(self, url):
        print("getting page {}".format(url))
        response = http.request('GET', url)

        # store text content
        paragraphs = justext.justext(response.data, justext.get_stoplist("Portuguese"))
        
        l = url.split('/')
        post_name = l[-1] if l[-1] != '' else l[-2]
        post_name = fl_name.split('.html')[0]
        fl_name = '{}_{}_{}'.format(l[3], l[4], post_name)
        
        with open("{}/{}.txt".format(self.corpus_path, fl_name), "w") as output_file:
            for paragraph in paragraphs:
                if not paragraph.is_boilerplate:
                    output_file.write(paragraph.text)
        
        # get links
        soup = BeautifulSoup(response.data, 'html.parser')
        links = [link.get('href') for link in soup.findAll('a', attrs={'href': re.compile(self.url_pattern)})]

        return links

    def add_links(self, links):
        links = list(set(links))
        self.to_be_visited.extend([link for link in links if link not in self.visited_links])

    def get_next_link(self):
        next_link = self.to_be_visited.pop(0)
        self.visited_links[next_link] = None
        return next_link

In [4]:
crawler_momentum = Crawler(
    '../data/corpora/blog/momentum_saga',
    'http://www.momentumsaga.com/2019/02/resenha-skyward-de-brandon-sanderson.html',
    '^http(s)?:\/\/www\.momentumsaga\.com\/\d{4}\/\d{2}\/.+\.html$'
)

crawler_momentum.crawl()

getting page http://www.momentumsaga.com/2019/02/resenha-skyward-de-brandon-sanderson.html
getting page http://www.momentumsaga.com/2014/09/resenha-trilogia-divergente-de-veronica-roth.html
getting page http://www.momentumsaga.com/2014/09/tag-cinquenta-fatos-sobre-mim.html
getting page http://www.momentumsaga.com/2018/02/dez-coisas-que-voce-nao-sabia-sobre-interestelar.html
getting page http://www.momentumsaga.com/2018/02/resenha-ordem-vermelha-filhos-da-degradacao-de-felipe-castilho.html
getting page http://www.momentumsaga.com/2018/02/colocando-seu-ebook-na-amazon-publicacao.html
getting page http://www.momentumsaga.com/2019/02/resenha-skyward-de-brandon-sanderson.html
getting page http://www.momentumsaga.com/2019/02/resenha-a-pequena-sereia-e-o-reino-das-ilusoes-de-louise-oneill.html
getting page http://www.momentumsaga.com/2019/02/como-ter-foco-e-finalizar-tarefas.html
getting page http://www.momentumsaga.com/2014/09/tag-cinquenta-fatos-sobre-mim.html
getting page http://www.moment

In [5]:
crawler_penumbra = Crawler(
    '../data/corpora/blog/penumbra_livros',
    'http://www.penumbralivros.com.br/2018/12/as-vantagens-de-praticar-magia-do-caos/',
    '^http(s)?:\/\/www\.penumbralivros\.com\.br\/\d{4}\/\d{2}\/\D+\/$'
)

crawler_penumbra.crawl()

getting page http://www.penumbralivros.com.br/2018/12/as-vantagens-de-praticar-magia-do-caos/
getting page http://www.penumbralivros.com.br/2019/03/entendendo-as-premonicoes-e-mudando-o-futuro/
getting page http://www.penumbralivros.com.br/2017/04/universo-quadridimensional-e-o-livro-da-lei/
getting page http://www.penumbralivros.com.br/2019/04/como-o-inconsciente-coletivo-pode-ser-fonte-de-magia/
getting page http://www.penumbralivros.com.br/2019/04/cuidado-grupo-internacional-ocultista-se-alimenta-da-depressao-e-busca-levar-bruxas-a-morte/
getting page http://www.penumbralivros.com.br/2019/03/encontrando-a-morte-diferentes-faces-de-uma-mesma-potencia/
getting page http://www.penumbralivros.com.br/2018/12/servidores-publicos-sera-que-funcionam/
getting page http://www.penumbralivros.com.br/2019/03/o-inconsciente-coletivo-como-fonte-de-materia-magicka-pt-i/
getting page http://www.penumbralivros.com.br/2018/12/seria-o-paganismo-moderno-apenas-uma-brincadeira/
getting page http://www.pe